# Data Analysis on Real World Data-set with SQL and Python
This code loads a dataset into a SQL database using Pandas Dataframes and performs some simple querries using SQL Magic

We will be using three datasets that are available on the city of Chicago's Data Portal:

Socioeconomic Indicators in Chicago: https://data.cityofchicago.org/Health-Human-Services/Census-Data-Selected-socioeconomic-indicators-in-C/kn9c-c2s2

Chicago Public Schools: https://data.cityofchicago.org/Education/Chicago-Public-Schools-Progress-Report-Cards-2011-/9xs2-f89t

Chicago Crime Data: https://data.cityofchicago.org/Public-Safety/Crimes-2001-to-present/ijzp-q8t2

In [ ]:
!pip install pandas
!pip install ipython-sql prettytable

import prettytable
prettytable.DEFAULT = 'DEFAULT'

In [ ]:
# First we install dependencies and connect to database 
import csv, sqlite3, pandas 

con = sqlite3.connect("FinalDB.db")
cur = con.cursor()

In [ ]:
%load_ext sql

%sql sqlite:///FinalDB.db

Using Pandas, we load the data available in the links above to dataframes. Then Use these dataframes to load data on to the database FinalDB.db as tables.

In [ ]:
df1 = pandas.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoCensusData.csv?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork20127838-2021-01-01")
df1.to_sql("CENSUS_DATA", con, if_exists="replace", index=False, method='multi')
df2 = pandas.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoPublicSchools.csv?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork20127838-2021-01-01")
df2.to_sql('CHICAGO_PUBLIC_SCHOOLS', con, if_exists='replace', index=False, method='multi')
df3 = pandas.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoCrimeData.csv?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork20127838-2021-01-01")
df3.to_sql('CHICAGO_CRIME_DATA', con, if_exists='replace', index=False, method='multi')
# three tables created: CENSUS_DATA, CHICAGO_PUBLIC_SCHOOLS, CHICAGO_CRIME_DATA

In [ ]:
%sql sqlite:///FinalDB.db

Now we Run some simple Queries on our data using SQL Magic 

In [ ]:
# Show all tables in our dataset
%sql SELECT name FROM sqlite_master WHERE type='table';

In [ ]:
# Total Number of Crime from Crime data Table 
%%sql
SELECT COUNT(*) AS total_crimes FROM CHICAGO_CRIME_DATA;

In [ ]:
# community area names and numbers with per capita income less than 11000.
%%sql 
SELECT COMMUNITY_AREA_NAME, COMMUNITY_AREA_NUMBER 
FROM CENSUS_DATA
WHERE PER_CAPITA_INCOME > 11000;

In [ ]:
# All crimes involvind a minor 
%sql SELECT * FROM CHICAGO_CRIME_DATA WHERE DESCRIPTION LIKE "%CHILD%"

In [ ]:
# CRIMES COMMITED AT SCHOOLS, NO REPITIONS 
%sql SELECT DISTINCT * FROM CHICAGO_CRIME_DATA WHERE LOCATION_DESCRIPTION LIKE '%SCHOOL%'

In [ ]:
# SCHOOL, TYPE AND avg Safety Score 
%%sql
SELECT 
    NAME_OF_SCHOOL, 
    "Elementary, Middle, or High School", 
    AVG(SAFETY_SCORE) AS AVG_SAFE
FROM CHICAGO_PUBLIC_SCHOOLS
WHERE "Elementary, Middle, or High School" = 'MS'
GROUP BY NAME_OF_SCHOOL, "Elementary, Middle, or High School";

In [ ]:
# top5 community areas with highest % of households below poverty line
%%sql
SELECT * 
FROM CENSUS_DATA
ORDER BY PERCENT_HOUSEHOLDS_BELOW_POVERTY DESC
LIMIT 5;

In [ ]:
# Most crime prone community area (i count the number of occurencies)
%%sql
SELECT COMMUNITY_AREA_NUMBER, COUNT(*) AS NUM_OCCURRENCES
FROM CHICAGO_CRIME_DATA
GROUP BY COMMUNITY_AREA_NUMBER
ORDER BY NUM_OCCURRENCES DESC
LIMIT 5;